In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import beta
from scipy.optimize import minimize

In [ ]:
# generate n random data from beta mixture model
def generate_beta_mixture(n, weights, a_params, b_params):
    """
    Generate n random samples from a beta mixture model.

    Parameters:
    n (int): Number of samples to generate.
    weights (list): List of weights for each beta component.
    a_params (list): List of 'a' parameters for each beta component.
    b_params (list): List of 'b' parameters for each beta component.

    Returns:
    np.ndarray: Array of generated samples.
    """
    # Normalize weights
    weights = np.array(weights)
    weights /= weights.sum()

    # Choose components based on weights
    components = np.random.choice(len(weights), size=n, p=weights)
    # print(components)
    # Generate samples
    samples = np.array([beta.rvs(a_params[i], b_params[i]) for i in components])

    return samples

def beta_mixture_likelihood(params, n_components, samples):
    """
    Calculate the negative log-likelihood of the beta mixture model.

    Parameters:
    params (list): List of parameters to optimize (weights, a_params, b_params).
    n_components (int): Number of components in the mixture model.
    samples (np.ndarray): Array of observed samples.

    Returns:
    float: Negative log-likelihood value.
    """
    likelihood = np.zeros_like(samples)

    for i in range(n_components):
        likelihood += params[i] * beta.pdf(samples, params[n_components + i], params[2 * n_components + i])

    # Avoid log(0) by adding a small constant
    likelihood = np.clip(likelihood, 1e-10, None)
    return -np.sum(np.log(likelihood))
def estimate_beta_mixture(samples, n_components):
    """ Estimate the parameters of a beta mixture model.
    Parameters:
    samples (np.ndarray): Array of observed samples.
    n_components (int): Number of components in the mixture model.
    Returns:
    list: List of estimated parameters (weights, a_params, b_params).
    """
    # Initialize parameters
    weights = np.ones(n_components) / n_components
    a_params = np.ones(n_components)
    b_params = np.ones(n_components)

    # Optimize the parameters
    params0 = np.concatenate((weights, a_params, b_params))
    # Set bounds for weights (0,1) and a,b parameters (>0)    
    bounds = [(0, 1)] * n_components + [(1e-2, None)] * n_components + [(1e-2, None)] * n_components
    # add constraint that weights sum to 1
    const = {'type': 'eq', 'fun': lambda x: np.sum(x[:n_components]) - 1}
    # set options
    options = {'maxiter': 10000, 'disp': False, 'ftol': 1e-6}
    res = minimize(beta_mixture_likelihood, params0, \
                   args=(n_components, samples), \
                    bounds=bounds, constraints=const, options=options)
    weights, a_params, b_params = res.x[:n_components], res.x[n_components:2*n_components], res.x[2*n_components:]

    return weights, a_params, b_params